# Lecture 8: The shooting method for BVP's

**Overview:**
* Root finding and boundary value problems.

**Next Lecture:**
* Shoot and Matching, 1D Schrodinger eqn.

---

### Tasks
- How does the solution for the launch angle depend on the initial bracketing window for your search?
- Can you extend this code to handle the case with air resistance? 
- Does the addition of air resistance result in new complications for the root finder?

In [2]:
#%matplotlib notebook
import numpy as np, matplotlib.pyplot as plt
from numpy import pi
import Particle3D as pt


In [13]:
def bisect(f, bracket, target, v, Cd, tol = 1.e-6):
    """ find the root of a function f using bisection
    a and b are low and high bracket limits
    v is a velocity
    Cd is a drag coefficent """
    a = bracket[0]
    b = bracket[1]
    
    fa = f(a,target, v, Cd)
    fb = f(b,target, v, Cd)
    gap  = abs(a - b)
   
    if fa == None :
        return 
    if fb == None :
        return
    
    if (fa*fb > 0.0):
        print('Bisection error: no root bracketed')
        return None
    elif fa == 0.0: return a
    elif fb == 0.0: return b

    while(True):
        xmid = 0.5*(a+b)
        fmid = f(xmid, target, v, Cd)

        if (fa*fmid > 0.0):
            a, fa = xmid, fmid
        else :b = xmid

        if (fmid == 0.0 or abs (b-a) < tol*gap): break

    return xmid

# the root of ft give the total time to reach target
def ft(t,target, V,Cd):
    # when air resistance is included you must define another function 
    # to find tfo
    
    p = pt.Projectile(tf = t,z0 = 0, v0 = V[0], u0 = V[1], w0 = V[2], Cd = Cd) 
    for ii in range(p.npoints):
        p.RK4_step()

    return p.x[1] - target[0]

# the root of fy is our solution
def fy(theta, target, v0, Cd):
    # only working in two dimensions (y,z) for now,.
    V = [0, v0*np.cos(theta), v0*np.sin(theta)]
    
    # if there is no drag, tf is easy to calculate analytically
    # with drag we must integrate and use a root finder to locate tf
    
    if Cd == 0.0:
        tf = target[0]/V[1]
    else:
        t_max = 10.0
        tf = bisect(ft,[0,t_max],target[0],V[1],Cd)

    # check if we can make the range
    if tf == None:
        print('Initial velocity will not cover range')
        return 
    
    nsteps = 100
    dt = tf/nsteps
    p = pt.Projectile(tf = tf, z0 = 0, v0 = V[0], u0 = V[1], w0 = V[2], dt = dt, Cd = Cd) 
    for ii in range(nsteps):
        p.RK4_step()
    
    print("y(tf) = ", p.x[2])
    return p.x[2] - target[1] 
    

* Change parameters and find launch angle below

In [14]:
v0 = 26 #initial launch speed
xb = 35 # x coordinate of target
yb = 6 # y coordinate of target
Cd = 10.00 # drag coefficient, you must edit ft and fy above to work for non-zero Cd

theta_bracket = [0.01,0.5] # in radians

# find the launch angle
# it would be more efficient to first check if we bracket the root before trying a full
# bisection search
theta = None
while not theta:
    theta = bisect(fy, theta_bracket, [xb, yb], v0, Cd)
    
    if not theta:
        print("Increasing launch speed by 1 %\n")
        v0=v0*1.01

# only make a plot if we can hit our target
if (theta != None):
    print("theta = ", theta*180/pi)
    # max time for plotting purposes
    t_max = xb/(v0*np.cos(theta))

    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    # plot the trajectory
    V = [0, v0*np.cos(theta), v0*np.sin(theta)]
    p = pt.Projectile(tf = t_max*2,  z0 = 0, u0 = V[0], v0 = V[1], w0 = V[2], dt = 0.001, Cd = Cd) 
    p.scipy_trajectory()
    ax.plot(p.xv[:,1], p.xv[:,2], lw = 2, label = '$v_0$ = {:.2f}, $\\theta$ = {:.1f}'.format(v0,theta*180/np.pi))

    # plot the target position
    ax.vlines(xb, 0 , yb)
    ax.plot(xb, yb, marker ='o', ls = 'None', color = 'r', label = 'Target')
    
    
    ax.set_xlabel('x (m)')
    ax.set_ylabel('y (m)')
    ax.legend()
    plt.show()

IndexError: invalid index to scalar variable.